# Step 1 - Prepare roads

This notebook merges roads/paths and prepares speeds based on their input data parameters. It is set up for transport networks in Bhutan but can be adapted to other contexts

In [7]:
import os, sys
import pandas as pd
import geopandas as gpd
import numpy as np
import pyproj as pyp

Projection

In [8]:
dest_crs = 'EPSG:32642'

#### Background: essential data prep

Extracting raster values to lines in python is cumbersome and for our purposes, pointlessly so\: the increased control yields no gains in accuracy. Therefore I prefer to join in average slopes per road segment in QGIS / ArcGIS ahead of time and process the resulting slope categories here. You can use the Add Surface Information tool in ArcGIS to accomplish (theoretically the SAGA toolkit has similar tools in QGIS but I wasn't able to maek them work). Compute the Average Slope for every layer this way.

#### Set parameters

In [9]:
# Production date for outputs being created

prod_date = '210923'
print(prod_date)

210923


In [10]:
input_pth = r'P:\PAK\GEO'
osm_pth = r'P:\PAK\\Code\Accessibility\osm_kpk_final_sep'
transport_pth = r'P:\PAK\\Code\Accessibility\Pak_Official_Roads_Combined'
ghs_pth = r'P:\PAK\\Code\Accessibility\GHS'

#### OSM data

In [11]:
# Load in the latest OSM data
# Assumes data to have been downloaded from Geofabrik to a folder with today's date

osm = gpd.read_file(os.path.join(osm_pth,'osm_kpk_roads_split.shp'))

In [12]:
# Rename Geofabrik's default 'flcass' column to the standard 'highway'
osm.rename({'fclass':'highway'},axis=1,inplace=True)

In [13]:
osm.highway.unique()

array(['tertiary', 'secondary', 'residential', 'trunk_link', 'trunk',
       'primary', 'service', 'unclassified', 'motorway', 'motorway_link',
       'footway', 'secondary_link', 'primary_link', 'tertiary_link',
       'living_street', 'pedestrian', 'track', 'cycleway', 'path',
       'track_grade3', 'track_grade2', 'track_grade4', 'unknown', 'steps',
       'track_grade5', 'track_grade1', 'bridleway'], dtype=object)

In [14]:
# dicts containing lists of values to replace, with the new key listed lasts

track_dct = dict.fromkeys(['track_grade1','track_grade2','track_grade3','track_grade4','track_grade5'], 'track')
minor_rd_dct = dict.fromkeys(['unclassified','road','service','residential', 'living_street'], 'minor_road')
pth_dct = dict.fromkeys(['path','footway','steps','pedestrian', 'bridleway'], 'path')

In [15]:
# Update the original dict with these new dicts
highway_replace_dct = {}

highway_replace_dct.update(track_dct)
highway_replace_dct.update(minor_rd_dct)
highway_replace_dct.update(pth_dct)

In [16]:
highway_replace_dct

{'track_grade1': 'track',
 'track_grade2': 'track',
 'track_grade3': 'track',
 'track_grade4': 'track',
 'track_grade5': 'track',
 'unclassified': 'minor_road',
 'road': 'minor_road',
 'service': 'minor_road',
 'residential': 'minor_road',
 'living_street': 'minor_road',
 'path': 'path',
 'footway': 'path',
 'steps': 'path',
 'pedestrian': 'path',
 'bridleway': 'path'}

In [17]:
# streamline highway values to a few key types using the above dictionary
osm['highway'] = osm['highway'].replace(highway_replace_dct)

In [18]:
osm.highway.unique()

array(['tertiary', 'secondary', 'minor_road', 'trunk_link', 'trunk',
       'primary', 'motorway', 'motorway_link', 'path', 'secondary_link',
       'primary_link', 'tertiary_link', 'track', 'cycleway', 'unknown'],
      dtype=object)

Filter out any lingering highway types we don't want using a list of values

In [19]:
accepted_road_types = ['path',\
                       'track','minor_road',\
                       'tertiary','secondary','primary','trunk','motorway',\
                       'tertiary_link','secondary_link','primary_link','trunk_link','motorway_link']

In [20]:
osm = osm[osm['highway'].isin(accepted_road_types)]

In [21]:
osm.highway.unique()

array(['tertiary', 'secondary', 'minor_road', 'trunk_link', 'trunk',
       'primary', 'motorway', 'motorway_link', 'path', 'secondary_link',
       'primary_link', 'tertiary_link', 'track'], dtype=object)

In [22]:
osm.head(2)

,fid_1,cat,osm_id,code,highway,name,ref,oneway,maxspeed,layer,bridge,tunnel,Z_Mean,Avg_Slope,geometry
0,1.0,1,5115246,5115,tertiary,سنبل روڈ,None,F,60,0,F,F,552.066479,3.293892,"LINESTRING (872158.150 3735598.793, 872148.766..."
1,2.0,2,5115247,5114,secondary,ناظم الدین روڈ,None,F,60,0,F,F,556.894392,3.491170,"LINESTRING (869490.900 3732937.943, 869510.249..."


Rename OSM data categories to gov categories

In [23]:
# dicts containing lists of values to replace, with the new key listed lasts

provincial_dct = dict.fromkeys(['primary','primary_link','trunk','trunk_link','motorway', 'motorway_link'], 'Provincial')
district_dct = dict.fromkeys(['secondary','secondary_link','tertiary','tertiary_link'], 'District')
access_dct = dict.fromkeys(['track'], 'Access')
collector_dct = dict.fromkeys(['minor_road'], 'Collector')

In [24]:
gov_align_dct = {}

gov_align_dct.update(provincial_dct)
gov_align_dct.update(district_dct)
gov_align_dct.update(access_dct)
gov_align_dct.update(collector_dct)

In [25]:
gov_align_dct

{'primary': 'Provincial',
 'primary_link': 'Provincial',
 'trunk': 'Provincial',
 'trunk_link': 'Provincial',
 'motorway': 'Provincial',
 'motorway_link': 'Provincial',
 'secondary': 'District',
 'secondary_link': 'District',
 'tertiary': 'District',
 'tertiary_link': 'District',
 'track': 'Access',
 'minor_road': 'Collector'}

In [26]:
osm['Road_Class'] = osm['highway'].map(gov_align_dct).fillna('Path')

In [27]:
osm.tail()

,fid_1,cat,osm_id,code,highway,name,ref,oneway,maxspeed,layer,bridge,tunnel,Z_Mean,Avg_Slope,geometry,Road_Class
232052,232053.0,195507,985249269,5134,secondary_link,None,None,B,0,0,F,F,553.429883,3.504753,"LINESTRING (875116.879 3737034.284, 875123.777...",District
232053,232054.0,195508,985249270,5134,secondary_link,None,None,B,0,0,F,F,557.172849,2.487046,"LINESTRING (875327.706 3737178.791, 875336.436...",District
232054,232055.0,195509,985267867,5154,path,None,None,B,0,0,F,F,541.991836,7.932720,"LINESTRING (871247.340 3733772.825, 871233.892...",Path
232055,232056.0,195510,985267868,5154,path,None,None,B,0,0,F,F,541.098468,1.150868,"LINESTRING (871234.968 3733568.472, 871227.958...",Path
232056,232057.0,195511,985267869,5154,path,None,None,B,0,0,F,F,534.694173,3.479288,"LINESTRING (871112.441 3733829.463, 871105.128...",Path


Spatial join on administrative information

In [28]:
#osm = gpd.sjoin(osm,adm2[['geometry','Dzongkhag','Gewog','adm1_code','adm2_code','hourly_wage']],how='left',op='within')

In [29]:
#osm.drop('index_right',axis=1,inplace=True)

In [30]:
#osm.head()

Slim down to just the key columns to join

In [31]:
osm[['Surface','MNT_Num','District_RONET','Road_Class_RONET','Surface1_RONET','Road_Condition_RONET','Surface2_RONET','TYPE']] = np.nan

In [32]:
#osm.head(2)

In [33]:
osm.head(2)

,fid_1,cat,osm_id,code,highway,name,ref,oneway,maxspeed,layer,...,geometry,Road_Class,Surface,MNT_Num,District_RONET,Road_Class_RONET,Surface1_RONET,Road_Condition_RONET,Surface2_RONET,TYPE
0,1.0,1,5115246,5115,tertiary,سنبل روڈ,None,F,60,0,...,"LINESTRING (872158.150 3735598.793, 872148.766...",District,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,2,5115247,5114,secondary,ناظم الدین روڈ,None,F,60,0,...,"LINESTRING (869490.900 3732937.943, 869510.249...",District,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
osm_slim = osm[['geometry','Road_Class','Surface','MNT_Num','District_RONET','Road_Class_RONET','Surface1_RONET','Road_Condition_RONET','Surface2_RONET','TYPE','Avg_Slope','Z_Mean']]

#### Gov data

In [35]:
gov = gpd.read_file(os.path.join(transport_pth,'combined_KPK_mnt_splt_utm.shp'))

In [36]:
#gov.rename({'GEWOG':'Gewog','DZONGKHAG':'Dzongkhag','SURFACETYP':'Surface','ROAD_CLASS':'Road_class'},axis=1,inplace=True)

In [37]:
gov.head(2)

,fid_1,cat,fid_12,Road_ID,Road_No,Road_Name,SegNo,Seg_Name,Road_Class,Pavement_T,...,MNT_RONE_1,MNT_RONE_2,MNT_Functy,MNT_Traf_1,MNT_Surfac,MNT_Condit,MNT_Final_,Z_Mean,Avg_Slope,geometry
0,1.0,1,1.0,1,S-1,Peshawar-Charsadda-Mardan-Swabi-Topi- Ghazi-Si...,1.0,"Peshawar,Charsadda Road(Dual carriage Way)",PKHA,Asphalt Concrete,...,None,None,None,None,None,None,None,301.968822,2.888695,"LINESTRING (753230.065 3782493.432, 753206.004..."
1,2.0,1,1.0,1,S-1,Peshawar-Charsadda-Mardan-Swabi-Topi- Ghazi-Si...,1.0,"Peshawar,Charsadda Road(Dual carriage Way)",PKHA,Asphalt Concrete,...,None,None,None,None,None,None,None,297.456491,2.627899,"LINESTRING (751720.466 3782845.841, 751693.010..."


Check out the data 

In [38]:
gov.Road_Class.unique()

array(['PKHA', 'B', 'A', None, 'S'], dtype=object)

In [39]:
gov.Pavement_T.unique()

array(['Asphalt Concrete', None, 'Gravel', 'Black Topping',
       'Triple Surface Treatment', 'AC/TST', 'A', 'C', 'E', 'RP',
       'RP = 10', 'RP= 1.5', 'RP = 2', 'RP= 1.25', 'RP= 1.50', 'RP = 1',
       'RP= 1', 'D', 'B', 'TSR', 'Shingle', 'F', 'R.P', 'SD', 'SG', 'DO'],
      dtype=object)

In [40]:
gov.MNT_RONET_.unique()

array([None, 'Good', 'Fair', 'Poor', 'poor'], dtype=object)

In [41]:
# dicts containing lists of values to replace, with the new key listed lasts

provi_dct = dict.fromkeys(['PKHA'], 'Provincial')
dis_dct = dict.fromkeys(['A','S'], 'District')
acc_dct = dict.fromkeys(['B'], 'Access')

In [42]:
new_gov_dct = {}

new_gov_dct.update(provi_dct)
new_gov_dct.update(dis_dct)
new_gov_dct.update(acc_dct)

In [43]:
new_gov_dct

{'PKHA': 'Provincial', 'A': 'District', 'S': 'District', 'B': 'Access'}

In [44]:
gov['Road_Class'] = gov['Road_Class'].replace(new_gov_dct).fillna('Collector')

In [45]:
gov.Road_Class.unique()

array(['Provincial', 'Access', 'District', 'Collector'], dtype=object)

In [46]:
gov.head(2)

,fid_1,cat,fid_12,Road_ID,Road_No,Road_Name,SegNo,Seg_Name,Road_Class,Pavement_T,...,MNT_RONE_1,MNT_RONE_2,MNT_Functy,MNT_Traf_1,MNT_Surfac,MNT_Condit,MNT_Final_,Z_Mean,Avg_Slope,geometry
0,1.0,1,1.0,1,S-1,Peshawar-Charsadda-Mardan-Swabi-Topi- Ghazi-Si...,1.0,"Peshawar,Charsadda Road(Dual carriage Way)",Provincial,Asphalt Concrete,...,None,None,None,None,None,None,None,301.968822,2.888695,"LINESTRING (753230.065 3782493.432, 753206.004..."
1,2.0,1,1.0,1,S-1,Peshawar-Charsadda-Mardan-Swabi-Topi- Ghazi-Si...,1.0,"Peshawar,Charsadda Road(Dual carriage Way)",Provincial,Asphalt Concrete,...,None,None,None,None,None,None,None,297.456491,2.627899,"LINESTRING (751720.466 3782845.841, 751693.010..."


In [47]:
gov.rename({'MNT_Distri':'District_RONET','MNT_Road_2':'Road_Class_RONET',\
            'MNT_Paveme':'Surface1_RONET','MNT_RONET_':'Road_Condition_RONET',\
            'MNT_RONE_2':'Surface2_RONET','Pavement_T':'Surface'},axis=1,inplace=True)


In [48]:
gov.head(2)

,fid_1,cat,fid_12,Road_ID,Road_No,Road_Name,SegNo,Seg_Name,Road_Class,Surface,...,MNT_RONE_1,Surface2_RONET,MNT_Functy,MNT_Traf_1,MNT_Surfac,MNT_Condit,MNT_Final_,Z_Mean,Avg_Slope,geometry
0,1.0,1,1.0,1,S-1,Peshawar-Charsadda-Mardan-Swabi-Topi- Ghazi-Si...,1.0,"Peshawar,Charsadda Road(Dual carriage Way)",Provincial,Asphalt Concrete,...,None,None,None,None,None,None,None,301.968822,2.888695,"LINESTRING (753230.065 3782493.432, 753206.004..."
1,2.0,1,1.0,1,S-1,Peshawar-Charsadda-Mardan-Swabi-Topi- Ghazi-Si...,1.0,"Peshawar,Charsadda Road(Dual carriage Way)",Provincial,Asphalt Concrete,...,None,None,None,None,None,None,None,297.456491,2.627899,"LINESTRING (751720.466 3782845.841, 751693.010..."


In [49]:
gov_slim = gov[['geometry','Road_Class','Surface','MNT_Num','District_RONET','Road_Class_RONET','Surface1_RONET','Road_Condition_RONET','Surface2_RONET','TYPE','Avg_Slope','Z_Mean']]

In [50]:
gov.dtypes


fid_1          float64
cat              int64
fid_12         float64
Road_ID         object
Road_No         object
                ...   
MNT_Condit      object
MNT_Final_      object
Z_Mean         float64
Avg_Slope      float64
geometry      geometry
Length: 61, dtype: object

### Merge datasets

Merge together the two slimmed down datasets into one master road file for analysis

In [51]:
pre_master = gpd.GeoDataFrame(pd.concat([gov_slim,osm_slim],ignore_index=True))

In [52]:
pre_master.head()

,geometry,Road_Class,Surface,MNT_Num,District_RONET,Road_Class_RONET,Surface1_RONET,Road_Condition_RONET,Surface2_RONET,TYPE,Avg_Slope,Z_Mean
0,"LINESTRING (753230.065 3782493.432, 753206.004...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,2.888695,301.968822
1,"LINESTRING (751720.466 3782845.841, 751693.010...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,2.627899,297.456491
2,"LINESTRING (750113.667 3781854.106, 750071.618...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,3.436315,297.611728
3,"LINESTRING (748578.407 3780675.967, 748529.082...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,2.529004,297.079688
4,"LINESTRING (747415.585 3779232.688, 747411.570...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,1.714421,298.843813


In [53]:
#Create Default Road Condition

In [54]:
pre_master.Surface.unique()

array(['Asphalt Concrete', None, 'Gravel', 'Black Topping',
       'Triple Surface Treatment', 'AC/TST', 'A', 'C', 'E', 'RP',
       'RP = 10', 'RP= 1.5', 'RP = 2', 'RP= 1.25', 'RP= 1.50', 'RP = 1',
       'RP= 1', 'D', 'B', 'TSR', 'Shingle', 'F', 'R.P', 'SD', 'SG', 'DO',
       nan], dtype=object)

In [55]:
pre_master.Surface1_RONET.unique()

array([None, 'Gravel', 'AC', 'TST', 'RP', 'GR', 'BT', 'PCC',
       'MobileAsphlt', 'M/A', 'Asphalt', 'Shingle', 'SD', 'Premix',
       'Kacha', 'Earth Work', 'Subgrade', nan], dtype=object)

In [56]:
pre_master.Road_Class.unique()

array(['Provincial', 'Access', 'District', 'Collector', 'Path'],
      dtype=object)

In [57]:
pre_master.Road_Condition_RONET.unique()

array([None, 'Good', 'Fair', 'Poor', 'poor', nan], dtype=object)

In [58]:
pre_master.Surface2_RONET.unique()

array([None, 'Gravel', 'Asphaltic Mix', 'Surface Treatment',
       'Cement Concrete', 'Earth', nan], dtype=object)

In [59]:
# dicts containing lists of values to replace, with the new key listed lasts

#good_dct = dict.fromkeys(['Asphaltic Mix'], 'Good')
#fair_dct = dict.fromkeys(['Cement Concrete','Earth'], 'Fair')
#poor_dct = dict.fromkeys(['Surface Treatment','Gravel'], 'Poor')

In [60]:
#road_cond_dct = {}

#road_cond_dct.update(good_dct)
#road_cond_dct.update(fair_dct)
#road_cond_dct.update(poor_dct)

In [61]:
#master['Temp_Road_Cond'] = master['Surface2_RONET'].map(road_cond_dct)

In [62]:
#Add GHS dataset to distinguish urban and rural datasets

In [63]:
ghs = gpd.read_file(os.path.join(ghs_pth,'ghs_smod_32642.shp'))

In [64]:
ghs.head()

,fid,ID_HDC_G0,NAME_MAIN,NAME_LIST,POP_2015,BU_2015,geometry
0,1.0,1.0,Honolulu,Honolulu; Waipahu; Pearl City; Aiea,512824.842119,80.749801,"POLYGON ((5799712.849 16680872.717, 5794211.57..."
1,2.0,2.0,Papeete,Papeete,91453.693810,14.568072,"POLYGON ((4860903.478 -17561458.847, 4862223.3..."
2,3.0,3.0,Santa Maria,Santa Maria,123184.613197,42.173721,"POLYGON ((1362330.458 16084121.636, 1361295.19..."
3,4.0,4.0,Monterey,Monterey,67761.478500,28.009914,"POLYGON ((1468544.073 15889517.172, 1469571.95..."
4,5.0,5.0,Santa Barbara,Santa Barbara,118777.131254,40.079154,"POLYGON ((1303782.553 16149142.120, 1301707.99..."


In [65]:
ghs = ghs.rename(columns={'fid':'Number'})

In [66]:
ghs.head()

,Number,ID_HDC_G0,NAME_MAIN,NAME_LIST,POP_2015,BU_2015,geometry
0,1.0,1.0,Honolulu,Honolulu; Waipahu; Pearl City; Aiea,512824.842119,80.749801,"POLYGON ((5799712.849 16680872.717, 5794211.57..."
1,2.0,2.0,Papeete,Papeete,91453.693810,14.568072,"POLYGON ((4860903.478 -17561458.847, 4862223.3..."
2,3.0,3.0,Santa Maria,Santa Maria,123184.613197,42.173721,"POLYGON ((1362330.458 16084121.636, 1361295.19..."
3,4.0,4.0,Monterey,Monterey,67761.478500,28.009914,"POLYGON ((1468544.073 15889517.172, 1469571.95..."
4,5.0,5.0,Santa Barbara,Santa Barbara,118777.131254,40.079154,"POLYGON ((1303782.553 16149142.120, 1301707.99..."


In [67]:
master = gpd.sjoin(pre_master, ghs, how = "left", op = "intersects")

In [71]:
master.head()

,geometry,Road_Class,Surface,MNT_Num,District_RONET,Road_Class_RONET,Surface1_RONET,Road_Condition_RONET,Surface2_RONET,TYPE,Avg_Slope,Z_Mean,index_right,Number,ID_HDC_G0,NAME_MAIN,NAME_LIST,POP_2015,BU_2015
0,"LINESTRING (753230.065 3782493.432, 753206.004...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,2.888695,301.968822,6256.0,6257.0,6257.0,Charsadda,Charsadda,432942.199814,4.102688
1,"LINESTRING (751720.466 3782845.841, 751693.010...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,2.627899,297.456491,6256.0,6257.0,6257.0,Charsadda,Charsadda,432942.199814,4.102688
2,"LINESTRING (750113.667 3781854.106, 750071.618...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,3.436315,297.611728,6256.0,6257.0,6257.0,Charsadda,Charsadda,432942.199814,4.102688
3,"LINESTRING (748578.407 3780675.967, 748529.082...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,2.529004,297.079688,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"LINESTRING (747415.585 3779232.688, 747411.570...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,1.714421,298.843813,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
#Save the file to check the spatial joints -- optional

In [70]:
#master.to_file('P:\PAK\Code\Accessibility\GHS\master_check_Aug28.gpkg',layer='master_layer',driver='GPKG')

### Manipulate datasets to arrive at final speeds

Slope information has already been joined in to the input shapefiles in a pre-processing step. Thefore, proceed to...

Using slope, generate terrain category 

In [72]:
# old

#master['Terrain'] = pd.cut(master['Z_Mean'], [-np.inf, 1500, 2299, np.inf], 
#                            labels = ['Plains', 'Hills', 'Mountains']) # change labels herelabels = ['Plains', 'Hills', 'Mountains']) # change labels here
# new

master['Terrain'] = pd.cut(master['Avg_Slope'], [-np.inf, 8, 16, np.inf], 
                           labels = ['Plains', 'Hills', 'Mountains']) # change labels herelabels = ['Plains', 'Hills', 'Mountains']) # change labels here

Generate dry season speed based on terrain category and road type

In [73]:
terrain_class_filter = [master['Terrain'].str.contains('Plains') & master['Road_Class'].str.contains('Provincial'),    
    master['Terrain'].str.contains('Plains') & master['Road_Class'].str.contains('District'),    
    master['Terrain'].str.contains('Plains') & master['Road_Class'].str.contains('Access'),    
    master['Terrain'].str.contains('Plains') & master['Road_Class'].str.contains('Collector'),
    master['Terrain'].str.contains('Hills') & master['Road_Class'].str.contains('Provincial'),    
    master['Terrain'].str.contains('Hills') & master['Road_Class'].str.contains('District'),    
    master['Terrain'].str.contains('Hills') & master['Road_Class'].str.contains('Access'),    
    master['Terrain'].str.contains('Hills') & master['Road_Class'].str.contains('Collector'),
    master['Terrain'].str.contains('Mountains') & master['Road_Class'].str.contains('Provincial'),    
    master['Terrain'].str.contains('Mountains') & master['Road_Class'].str.contains('District'),    
    master['Terrain'].str.contains('Mountains') & master['Road_Class'].str.contains('Access'),    
    master['Terrain'].str.contains('Mountains') & master['Road_Class'].str.contains('Collector')]
                        

In [74]:
# Corresponding list of speeds, with line breaks by terrain type for editing and readability
speeds_lst = [80,50,40,20,\
                 60,40,30,15,\
                 40,30,20,10]

In [75]:
master['base_speed'] = np.select(terrain_class_filter,speeds_lst,default=0.5) # very low default as path speeds will be re-calculated separately using terrain down the road

In [76]:
# must convert the Terrain CategoricalDType to String to export as a geopackage
master['Terrain'] = master['Terrain'].astype(str)

In [77]:
master.head()

,geometry,Road_Class,Surface,MNT_Num,District_RONET,Road_Class_RONET,Surface1_RONET,Road_Condition_RONET,Surface2_RONET,TYPE,...,Z_Mean,index_right,Number,ID_HDC_G0,NAME_MAIN,NAME_LIST,POP_2015,BU_2015,Terrain,base_speed
0,"LINESTRING (753230.065 3782493.432, 753206.004...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,301.968822,6256.0,6257.0,6257.0,Charsadda,Charsadda,432942.199814,4.102688,Plains,80.0
1,"LINESTRING (751720.466 3782845.841, 751693.010...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,297.456491,6256.0,6257.0,6257.0,Charsadda,Charsadda,432942.199814,4.102688,Plains,80.0
2,"LINESTRING (750113.667 3781854.106, 750071.618...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,297.611728,6256.0,6257.0,6257.0,Charsadda,Charsadda,432942.199814,4.102688,Plains,80.0
3,"LINESTRING (748578.407 3780675.967, 748529.082...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,297.079688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Plains,80.0
4,"LINESTRING (747415.585 3779232.688, 747411.570...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,298.843813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Plains,80.0


Reclassify various surface types into simpler categories

In [78]:
master.Surface2_RONET.unique()

array([None, 'Gravel', 'Asphaltic Mix', 'Surface Treatment',
       'Cement Concrete', 'Earth', nan], dtype=object)

In [79]:
master.Road_Class.unique()

array(['Provincial', 'Access', 'District', 'Collector', 'Path'],
      dtype=object)

In [80]:
master['Temp_Surface'] = np.where((master['Road_Class'] =='Provincial'),master['Road_Class'],master['Surface'])

In [81]:
master.Temp_Surface.unique()

array(['Provincial', 'C', 'E', 'A', 'RP', 'RP = 10', 'RP= 1.5', 'RP = 2',
       'RP= 1.25', 'RP= 1.50', 'RP = 1', 'RP= 1', 'D', 'B', 'TSR', None,
       'Shingle', 'F', 'R.P', 'SD', 'SG', 'DO', nan], dtype=object)

In [82]:
master.Surface.unique()

array(['Asphalt Concrete', None, 'Gravel', 'Black Topping',
       'Triple Surface Treatment', 'AC/TST', 'A', 'C', 'E', 'RP',
       'RP = 10', 'RP= 1.5', 'RP = 2', 'RP= 1.25', 'RP= 1.50', 'RP = 1',
       'RP= 1', 'D', 'B', 'TSR', 'Shingle', 'F', 'R.P', 'SD', 'SG', 'DO',
       nan], dtype=object)

In [83]:
master['Temp_Surface_Consolidated'] = np.where((master['Surface2_RONET'].isnull()==False),master['Surface2_RONET'],master['Temp_Surface'])

In [84]:
master.head(2)

,geometry,Road_Class,Surface,MNT_Num,District_RONET,Road_Class_RONET,Surface1_RONET,Road_Condition_RONET,Surface2_RONET,TYPE,...,Number,ID_HDC_G0,NAME_MAIN,NAME_LIST,POP_2015,BU_2015,Terrain,base_speed,Temp_Surface,Temp_Surface_Consolidated
0,"LINESTRING (753230.065 3782493.432, 753206.004...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,6257.0,6257.0,Charsadda,Charsadda,432942.199814,4.102688,Plains,80.0,Provincial,Provincial
1,"LINESTRING (751720.466 3782845.841, 751693.010...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,6257.0,6257.0,Charsadda,Charsadda,432942.199814,4.102688,Plains,80.0,Provincial,Provincial


In [85]:
master.Temp_Surface_Consolidated.unique()

array(['Provincial', 'C', 'Gravel', 'Asphaltic Mix', 'Surface Treatment',
       'E', 'Cement Concrete', 'RP', 'RP= 1.5', 'RP= 1.25', 'RP = 1',
       'RP= 1', 'A', 'D', 'TSR', None, 'F', 'SD', 'DO', 'Earth', nan],
      dtype=object)

In [86]:
#Reclassify Temp_Surface_Consolidated categories

In [87]:
pave_dct = dict.fromkeys(['Provincial','Black Topping','Triple Surface Treatment','Cement Concrete','AC/TST'],'Paved')
gravel_dct = dict.fromkeys(['Asphaltic Mix','Surface Treatment','TSR','A','C','D','DO','RP','RP= 1.5','RP= 1.25','RP = 1','RP= 1','Gravel'],'Gravel')
earth_dct = dict.fromkeys(['E','F','SD','Earth','None','nan'],'Earthen')

In [88]:
master.Temp_Surface_Consolidated = master.Temp_Surface_Consolidated.replace(pave_dct)
master.Temp_Surface_Consolidated = master.Temp_Surface_Consolidated.replace(gravel_dct)
master.Temp_Surface_Consolidated = master.Temp_Surface_Consolidated.replace(earth_dct)
master.Temp_Surface_Consolidated = master.Temp_Surface_Consolidated.fillna('Earthen')

In [89]:
master.Temp_Surface_Consolidated.unique()

array(['Paved', 'Gravel', 'Earthen'], dtype=object)

In [90]:
master['Surface_Final'] = np.where((master['NAME_MAIN'].isnull()==False), 'Paved', master['Temp_Surface_Consolidated'])

In [91]:
master.Surface_Final.unique()

array(['Paved', 'Gravel', 'Earthen'], dtype=object)

In [92]:
master.head(2)

,geometry,Road_Class,Surface,MNT_Num,District_RONET,Road_Class_RONET,Surface1_RONET,Road_Condition_RONET,Surface2_RONET,TYPE,...,ID_HDC_G0,NAME_MAIN,NAME_LIST,POP_2015,BU_2015,Terrain,base_speed,Temp_Surface,Temp_Surface_Consolidated,Surface_Final
0,"LINESTRING (753230.065 3782493.432, 753206.004...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,6257.0,Charsadda,Charsadda,432942.199814,4.102688,Plains,80.0,Provincial,Paved,Paved
1,"LINESTRING (751720.466 3782845.841, 751693.010...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,6257.0,Charsadda,Charsadda,432942.199814,4.102688,Plains,80.0,Provincial,Paved,Paved


In [93]:
master.Surface1_RONET.unique()

array([None, 'Gravel', 'AC', 'TST', 'RP', 'GR', 'BT', 'PCC',
       'MobileAsphlt', 'M/A', 'Asphalt', 'Shingle', 'SD', 'Premix',
       'Kacha', 'Earth Work', 'Subgrade', nan], dtype=object)

In [94]:
master.Surface2_RONET.unique()

array([None, 'Gravel', 'Asphaltic Mix', 'Surface Treatment',
       'Cement Concrete', 'Earth', nan], dtype=object)

In [95]:
master.Road_Condition_RONET.unique()

array([None, 'Good', 'Fair', 'Poor', 'poor', nan], dtype=object)

In [96]:
#paved1_dct = dict.fromkeys(['AC','MobileAsphlt','Asphalt','M/A','PCC'],'Paved')
#gravel1_dct = dict.fromkeys(['Gravel','TST','GR','RP','BT','Premix'],'Gravel')
#earth1_dct = dict.fromkeys(['Kacha','Subgrade','Earth Work','SD','Shingle','None','nan'],'Earthen')

In [97]:
#master.Surface1_RONET = master.Surface1_RONET.replace(paved1_dct)
#master.Surface1_RONET = master.Surface1_RONET.replace(gravel1_dct)
#master.Surface1_RONET = master.Surface1_RONET.replace(earth1_dct)
#master.Surface1_RONET = master.Surface1_RONET.fillna('Earthen')

Road condition based on Road_Class and Terrain

In [98]:
master['Terrain'] = master['Terrain'].astype(str)

In [99]:
road_condition_filter1 = [master['Terrain'].str.contains('Plains') & master['Road_Class'].str.contains('Provincial'),    
    master['Terrain'].str.contains('Plains') & master['Road_Class'].str.contains('District'),    
    master['Terrain'].str.contains('Plains') & master['Road_Class'].str.contains('Access'),    
    master['Terrain'].str.contains('Plains') & master['Road_Class'].str.contains('Collector'),
    master['Terrain'].str.contains('Hills') & master['Road_Class'].str.contains('Provincial'),    
    master['Terrain'].str.contains('Hills') & master['Road_Class'].str.contains('District'),    
    master['Terrain'].str.contains('Hills') & master['Road_Class'].str.contains('Access'),    
    master['Terrain'].str.contains('Hills') & master['Road_Class'].str.contains('Collector'),
    master['Terrain'].str.contains('Mountains') & master['Road_Class'].str.contains('Provincial'),    
    master['Terrain'].str.contains('Mountains') & master['Road_Class'].str.contains('District'),    
    master['Terrain'].str.contains('Mountains') & master['Road_Class'].str.contains('Access'),    
    master['Terrain'].str.contains('Mountains') & master['Road_Class'].str.contains('Collector')]

In [100]:
default_cond1 = ['Good','Poor','Poor','Poor',\
                 'Good','Poor','Poor','Good',\
                 'Fair','Poor','Poor','Poor']

In [101]:
master['default_road_cond1'] = np.select(road_condition_filter1, default_cond1)

In [102]:
master.dtypes


geometry                     geometry
Road_Class                     object
Surface                        object
MNT_Num                        object
District_RONET                 object
Road_Class_RONET               object
Surface1_RONET                 object
Road_Condition_RONET           object
Surface2_RONET                 object
TYPE                           object
Avg_Slope                     float64
Z_Mean                        float64
index_right                   float64
Number                        float64
ID_HDC_G0                     float64
NAME_MAIN                      object
NAME_LIST                      object
POP_2015                      float64
BU_2015                       float64
Terrain                        object
base_speed                    float64
Temp_Surface                   object
Temp_Surface_Consolidated      object
Surface_Final                  object
default_road_cond1             object
dtype: object

In [103]:
# Road Condition based on Road Surface (Surface2_RONET) and Terrain

In [104]:
master['Terrain'] = master['Terrain'].astype(str)

In [105]:
road_condition_filter2 = [master['Terrain'].str.contains('Plains') & master['Surface2_RONET'].str.contains('Asphaltic Mix'),    
    master['Terrain'].str.contains('Plains') & master['Surface2_RONET'].str.contains('Cement Concrete'),
    master['Terrain'].str.contains('Plains') & master['Surface2_RONET'].str.contains('Earthen'),                      
    master['Terrain'].str.contains('Plains') & master['Surface2_RONET'].str.contains('Gravel'),
    master['Terrain'].str.contains('Plains') & master['Surface2_RONET'].str.contains('Surface Treatment'),
    master['Terrain'].str.contains('Hills') & master['Surface2_RONET'].str.contains('Asphaltic Mix'),                      
    master['Terrain'].str.contains('Hills') & master['Surface2_RONET'].str.contains('Cement Concrete'),
    master['Terrain'].str.contains('Hills') & master['Surface2_RONET'].str.contains('Earthen'),                       
    master['Terrain'].str.contains('Hills') & master['Surface2_RONET'].str.contains('Gravel'),
    master['Terrain'].str.contains('Hills') & master['Surface2_RONET'].str.contains('Surface Treatment'),                     
    master['Terrain'].str.contains('Mountains') & master['Surface2_RONET'].str.contains('Asphaltic Mix'),                      
    master['Terrain'].str.contains('Mountains') & master['Surface2_RONET'].str.contains('Cement Concrete'),
    master['Terrain'].str.contains('Mountains') & master['Surface2_RONET'].str.contains('Earthen'),                        
    master['Terrain'].str.contains('Mountains') & master['Surface2_RONET'].str.contains('Gravel'),
    master['Terrain'].str.contains('Mountains') & master['Surface2_RONET'].str.contains('Surface Treatment')]                      

In [106]:
default_cond2 = ['Good','Poor','Poor','Poor','Poor',\
                 'Good','Poor','None','Poor','Poor',\
                 'Fair','Poor','None','Poor','Fair']

In [107]:
master['default_road_cond2'] = np.select(road_condition_filter2, default_cond2, default = None)

In [108]:
master.default_road_cond2.unique()

array([None, 'Poor', 'Good', 'Fair'], dtype=object)

In [109]:
# overwrite the default_cond1 with default_cond2

In [110]:
master['Temp_Road_Cond'] = np.where((master['default_road_cond2'].isnull()==False),master['default_road_cond2'],master['default_road_cond1'])

In [111]:
#overwrite Temp_Road_Cond with Road_Condition_RONET to arrive at "Road_Cond_Final"

In [112]:
master.Temp_Road_Cond.unique()

array(['Good', 'Fair', 'Poor', '0'], dtype=object)

In [113]:
master.head()

,geometry,Road_Class,Surface,MNT_Num,District_RONET,Road_Class_RONET,Surface1_RONET,Road_Condition_RONET,Surface2_RONET,TYPE,...,POP_2015,BU_2015,Terrain,base_speed,Temp_Surface,Temp_Surface_Consolidated,Surface_Final,default_road_cond1,default_road_cond2,Temp_Road_Cond
0,"LINESTRING (753230.065 3782493.432, 753206.004...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,432942.199814,4.102688,Plains,80.0,Provincial,Paved,Paved,Good,None,Good
1,"LINESTRING (751720.466 3782845.841, 751693.010...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,432942.199814,4.102688,Plains,80.0,Provincial,Paved,Paved,Good,None,Good
2,"LINESTRING (750113.667 3781854.106, 750071.618...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,432942.199814,4.102688,Plains,80.0,Provincial,Paved,Paved,Good,None,Good
3,"LINESTRING (748578.407 3780675.967, 748529.082...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,NaN,NaN,Plains,80.0,Provincial,Paved,Paved,Good,None,Good
4,"LINESTRING (747415.585 3779232.688, 747411.570...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,NaN,NaN,Plains,80.0,Provincial,Paved,Paved,Good,None,Good


In [114]:
master.Road_Condition_RONET.unique()

array([None, 'Good', 'Fair', 'Poor', 'poor', nan], dtype=object)

In [115]:
master['Road_Condition_RONET'] = np.where(master['Road_Condition_RONET'] == 'poor','Poor', master['Road_Condition_RONET'])

In [116]:
master['Road_Cond_Final'] = np.where((master['Road_Condition_RONET'].isnull()==False),master['Road_Condition_RONET'],master['Temp_Road_Cond'])

In [117]:
master.Road_Cond_Final.unique()

array(['Good', 'Fair', 'Poor', '0'], dtype=object)

In [118]:
master.head()

,geometry,Road_Class,Surface,MNT_Num,District_RONET,Road_Class_RONET,Surface1_RONET,Road_Condition_RONET,Surface2_RONET,TYPE,...,BU_2015,Terrain,base_speed,Temp_Surface,Temp_Surface_Consolidated,Surface_Final,default_road_cond1,default_road_cond2,Temp_Road_Cond,Road_Cond_Final
0,"LINESTRING (753230.065 3782493.432, 753206.004...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,4.102688,Plains,80.0,Provincial,Paved,Paved,Good,None,Good,Good
1,"LINESTRING (751720.466 3782845.841, 751693.010...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,4.102688,Plains,80.0,Provincial,Paved,Paved,Good,None,Good,Good
2,"LINESTRING (750113.667 3781854.106, 750071.618...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,4.102688,Plains,80.0,Provincial,Paved,Paved,Good,None,Good,Good
3,"LINESTRING (748578.407 3780675.967, 748529.082...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,NaN,Plains,80.0,Provincial,Paved,Paved,Good,None,Good,Good
4,"LINESTRING (747415.585 3779232.688, 747411.570...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,NaN,Plains,80.0,Provincial,Paved,Paved,Good,None,Good,Good


### Calculate speeds

In [119]:
speed_adj_class_filter = [master['Surface_Final'].str.contains('Earthen') & master['Road_Cond_Final'].str.contains('Good'),    
    master['Surface_Final'].str.contains('Earthen') & master['Road_Cond_Final'].str.contains('Fair'),    
    master['Surface_Final'].str.contains('Earthen') & master['Road_Cond_Final'].str.contains('Poor'),
    master['Surface_Final'].str.contains('Gravel') & master['Road_Cond_Final'].str.contains('Good'),    
    master['Surface_Final'].str.contains('Gravel') & master['Road_Cond_Final'].str.contains('Fair'),    
    master['Surface_Final'].str.contains('Gravel') & master['Road_Cond_Final'].str.contains('Poor'),
    master['Surface_Final'].str.contains('Paved') & master['Road_Cond_Final'].str.contains('Good'),    
    master['Surface_Final'].str.contains('Paved') & master['Road_Cond_Final'].str.contains('Fair'),    
    master['Surface_Final'].str.contains('Paved') & master['Road_Cond_Final'].str.contains('Poor')]

Dry season modeling: dry season speeds should also be (less dramatically) modified by the surface/class combination.

In [120]:
# Corresponding list of dry season speed modifiers, with line breaks by terrain type for editing and readability

dry_mods_lst = [ 0.6, 0.5, 0.4,\
                 0.75, 0.6, 0.5,\
                 1, 0.8, 0.6 ]

master['dry_mod'] = np.select(speed_adj_class_filter,dry_mods_lst,default=0.5) # very low default as path speeds will be re-calculated separately using terrain down the road
master['dry_speed'] = master.dry_mod * master.base_speed

Generate monsoon season speed based on terrain category, road type, and surface

In [121]:
# Corresponding list of monsoon speed modifiers, with line breaks by terrain type for editing and readability

msn_mods_lst = [ 0.4, 0.3, 0.2,\
                 0.7, 0.5, 0.4,\
                 0.9, 0.75, 0.5 ]


In [122]:
master['msn_mod'] = np.select(speed_adj_class_filter,msn_mods_lst,default=0.5) 
# very low default as path speeds will be re-calculated separately using terrain down the road

In [123]:
master['msn_speed'] = master.msn_mod * master.base_speed

In [124]:
master.head()

,geometry,Road_Class,Surface,MNT_Num,District_RONET,Road_Class_RONET,Surface1_RONET,Road_Condition_RONET,Surface2_RONET,TYPE,...,Temp_Surface_Consolidated,Surface_Final,default_road_cond1,default_road_cond2,Temp_Road_Cond,Road_Cond_Final,dry_mod,dry_speed,msn_mod,msn_speed
0,"LINESTRING (753230.065 3782493.432, 753206.004...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,Paved,Paved,Good,None,Good,Good,1.0,80.0,0.9,72.0
1,"LINESTRING (751720.466 3782845.841, 751693.010...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,Paved,Paved,Good,None,Good,Good,1.0,80.0,0.9,72.0
2,"LINESTRING (750113.667 3781854.106, 750071.618...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,Paved,Paved,Good,None,Good,Good,1.0,80.0,0.9,72.0
3,"LINESTRING (748578.407 3780675.967, 748529.082...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,Paved,Paved,Good,None,Good,Good,1.0,80.0,0.9,72.0
4,"LINESTRING (747415.585 3779232.688, 747411.570...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,Paved,Paved,Good,None,Good,Good,1.0,80.0,0.9,72.0


Winter Speed Modelling

In [125]:
# List of winter speed modifiers, with line breaks by terrain type for editing and readability
winter_mods_lst = [ 0.4, 0.3, 0.2,\
                    0.7, 0.5, 0.4,\
                    1, 0.75, 0.5 ]


In [126]:
master['winter_mod'] = np.select(speed_adj_class_filter,winter_mods_lst,default=0.5) 

In [130]:
# Define which winter mods to apply to which road based on their terrain

winter_cutoff_lst = [master['Terrain'] == 'Plains', \
                     master['Terrain'] == 'Hills', \
                     master['Terrain'] == 'Mountains']

In [127]:
# Corresponding list of monsoon speed modifiers, with line breaks by terrain type for editing and readability

winter_mods_lst = [ 0.4, 0.3, 0.2,\
                    0.7, 0.5, 0.4,\
                    1, 0.75, 0.5,
                    1, 0.9, 0.75]

In [128]:
winter_mod_revised =  [ master['dry_speed'], \
                       (master['dry_speed'] * (master['winter_mod'] + (1 - master['winter_mod']) /2) ), \
                       (master['base_speed'] * master['winter_mod'])]

In [131]:
master['winter_speed'] = np.select(winter_cutoff_lst, winter_mod_revised, default=master['dry_speed'])

In [132]:
master['winter_speed_final'] = np.where((master['TYPE'] == 'Snow Bound'), (master['dry_speed'] * master['winter_mod']), master['winter_speed'])

In [133]:
master.head(2)

,geometry,Road_Class,Surface,MNT_Num,District_RONET,Road_Class_RONET,Surface1_RONET,Road_Condition_RONET,Surface2_RONET,TYPE,...,default_road_cond2,Temp_Road_Cond,Road_Cond_Final,dry_mod,dry_speed,msn_mod,msn_speed,winter_mod,winter_speed,winter_speed_final
0,"LINESTRING (753230.065 3782493.432, 753206.004...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,None,Good,Good,1.0,80.0,0.9,72.0,1.0,80.0,80.0
1,"LINESTRING (751720.466 3782845.841, 751693.010...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,None,Good,Good,1.0,80.0,0.9,72.0,1.0,80.0,80.0


In [119]:
master.head(2)

,geometry,Road_Class,Surface,MNT_Num,District_RONET,Road_Class_RONET,Surface1_RONET,Road_Condition_RONET,Surface2_RONET,TYPE,...,default_road_cond1,default_road_cond2,Temp_Road_Cond,Road_Cond_Final,msn_mod,msn_speed,winter_mod,winter_speed,winter_speed_final,dry_mod
0,"LINESTRING (753230.065 3782493.432, 753206.004...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,Good,None,Good,Good,0.9,72.0,1.0,80.0,80.0,1.0
1,"LINESTRING (751720.466 3782845.841, 751693.010...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,Good,None,Good,Good,0.9,72.0,1.0,80.0,80.0,1.0


Checking final outputs before export

In [134]:
master[master['Terrain'] =='Mountains']

,geometry,Road_Class,Surface,MNT_Num,District_RONET,Road_Class_RONET,Surface1_RONET,Road_Condition_RONET,Surface2_RONET,TYPE,...,default_road_cond2,Temp_Road_Cond,Road_Cond_Final,dry_mod,dry_speed,msn_mod,msn_speed,winter_mod,winter_speed,winter_speed_final
64,"LINESTRING (803050.003 3833579.085, 803046.090...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,None,Fair,Fair,0.8,32.00,0.75,30.00,0.75,30.00,30.00
65,"LINESTRING (801642.023 3834462.783, 801641.695...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,None,Fair,Fair,0.8,32.00,0.75,30.00,0.75,30.00,30.00
82,"LINESTRING (835178.030 3824028.765, 835177.099...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,None,Fair,Fair,0.8,32.00,0.75,30.00,0.75,30.00,30.00
85,"LINESTRING (834238.919 3828953.400, 834223.838...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,None,Fair,Fair,0.8,32.00,0.75,30.00,0.75,30.00,30.00
93,"LINESTRING (834131.623 3837107.352, 834131.250...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,None,Fair,Fair,0.8,32.00,0.75,30.00,0.75,30.00,30.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240490,"LINESTRING (571285.317 3579510.650, 571279.645...",Path,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,0,0,0.5,0.25,0.50,0.25,0.50,0.25,0.25
240573,"LINESTRING (591330.750 3664537.974, 591334.168...",Collector,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Poor,Poor,0.4,4.00,0.20,2.00,0.20,2.00,2.00
240581,"LINESTRING (591246.055 3666434.685, 591263.459...",Collector,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Poor,Poor,0.4,4.00,0.20,2.00,0.20,2.00,2.00
241005,"LINESTRING (862976.543 3717220.595, 862979.892...",Access,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Poor,Poor,0.4,8.00,0.20,4.00,0.20,4.00,4.00


In [135]:
master[master['Terrain']=='Hills']

,geometry,Road_Class,Surface,MNT_Num,District_RONET,Road_Class_RONET,Surface1_RONET,Road_Condition_RONET,Surface2_RONET,TYPE,...,default_road_cond2,Temp_Road_Cond,Road_Cond_Final,dry_mod,dry_speed,msn_mod,msn_speed,winter_mod,winter_speed,winter_speed_final
39,"LINESTRING (815386.452 3809784.883, 815394.148...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,None,Good,Good,1.0,60.0,0.9,54.0,1.0,60.0,60.0
42,"LINESTRING (818781.628 3812914.787, 818786.523...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,None,Good,Good,1.0,60.0,0.9,54.0,1.0,60.0,60.0
43,"LINESTRING (820018.241 3813896.279, 820009.641...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,None,Good,Good,1.0,60.0,0.9,54.0,1.0,60.0,60.0
48,"LINESTRING (818840.316 3821663.157, 818818.959...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,None,Good,Good,1.0,60.0,0.9,54.0,1.0,60.0,60.0
48,"LINESTRING (818840.316 3821663.157, 818818.959...",Provincial,Asphalt Concrete,None,None,None,None,None,None,All Weather,...,None,Good,Good,1.0,60.0,0.9,54.0,1.0,60.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241270,"LINESTRING (382654.462 3442053.690, 382637.390...",District,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Poor,Poor,0.4,16.0,0.2,8.0,0.2,9.6,9.6
241271,"LINESTRING (381641.704 3441805.384, 381647.392...",District,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Poor,Poor,0.4,16.0,0.2,8.0,0.2,9.6,9.6
241282,"LINESTRING (369272.584 3448352.161, 369249.847...",District,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Poor,Poor,0.4,16.0,0.2,8.0,0.2,9.6,9.6
241318,"LINESTRING (865482.935 3724191.522, 865490.612...",Collector,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Good,Good,0.6,9.0,0.4,6.0,0.4,6.3,6.3


## Export final data

Export

In [136]:
master.to_file('P:\PAK\Code\Accessibility\Inputs\master_transport_Oct19.gpkg',layer='master_transport_all',driver='GPKG')
#master_rds.to_file('P:\PAK\Code\Accessibility\Inputs\master_transport.gpkg',layer='master_roads',driver='GPKG')
#master_pths.to_file('P:\PAK\Code\Accessibility\Inputs\master_transport.gpkg',layer='master_paths',driver='GPKG')


In [137]:
master.to_csv('P:\PAK\Code\Accessibility\Inputs\master_transport_Oct19.csv')